# Practice PS06: Recommendations engines (interactions-based)

For this assignment we will build and apply an item-based and model-based collaborative filtering recommenders for movies. 

Author: <font color="blue">Bernat Quintilla Castellón</font>

E-mail: <font color="blue">bernat.quintilla01@estudiant.upf.edu</font>

Date: <font color="blue">The current date here</font>

# 1. The Movies dataset

We will use the same dataset as in ps05, the 25M version of [MovieLens DataSet](https://grouplens.org/datasets/movielens/) released in late 2019. We will use a sub-set containing only movies released in the 2000s, and only 10% of the users and all of their ratings.

* **MOVIES** are described in `movies-2000s.csv` in the following format: `movieId,title,genres`.
* **RATINGS** are contained in `ratings-2000s.csv` in the following format: `userId,movieId,rating`
* **TAGS** are contained in `tags.csv` in the following format: `userId,movieId,tag,timestamp`

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

# 1.1. Load the input files

In [1]:
# Leave this code as-is

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from math import*
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import linear_kernel

In [2]:
# Leave this code as-is

FILENAME_MOVIES = "movies-2000s.csv"
FILENAME_RATINGS = "ratings-2000s.csv"
FILENAME_TAGS = "tags-2000s.csv"

In [3]:
# Leave this code as-is

movies = pd.read_csv(FILENAME_MOVIES, 
                    sep=',', 
                    engine='python', 
                    encoding='latin-1',
                    names=['movie_id', 'title', 'genres'])
display(movies.head(5))

ratings_raw = pd.read_csv(FILENAME_RATINGS, 
                    sep=',', 
                    encoding='latin-1',
                    engine='python',
                    names=['user_id', 'movie_id', 'rating'])
display(ratings_raw.head(5))

,movie_id,title,genres
0,2769,"Yards, The (2000)",Crime|Drama
1,3177,Next Friday (2000),Comedy
2,3190,Supernova (2000),Adventure|Sci-Fi|Thriller
3,3225,Down to You (2000),Comedy|Romance
4,3228,Wirey Spindell (2000),Comedy


,user_id,movie_id,rating
0,4,1,3.0
1,4,260,3.5
2,4,296,4.0
3,4,541,4.5
4,4,589,4.0


# 1.2. Merge the data into a single dataframe

Join the data into a single dataframe that should contain columns: user_id, movie_id, rating, timestamp, title, genders.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code from the previous practice that joined these three dataframes using "merge" into a single dataframe named "ratings". Print the first 5 rows of the resulting dataframe, which should contain columns "user_id", "movie_id", "rating", "title", and "genres".</font>

In [4]:
ratings = pd.merge(ratings_raw, movies)

# Print the first 5 rows of ratings
ratings.head(5)

,user_id,movie_id,rating,title,genres
0,4,3624,2.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western
1,152,3624,3.0,Shanghai Noon (2000),Action|Adventure|Comedy|Western
2,171,3624,3.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western
3,276,3624,4.0,Shanghai Noon (2000),Action|Adventure|Comedy|Western
4,494,3624,3.5,Shanghai Noon (2000),Action|Adventure|Comedy|Western


<font size="+1" color="red">Replace this cell with your code from the previous practice for "find_movies" that list movies containing a keyword</font>

In [5]:
def find_movies(keyword, movies):
    found_movies = []

    for index, row in movies.iterrows():
        if keyword in row['title']: #If keyword appears append row
            found_movies.append((row['movie_id'], row['title']))

    for movie_id, title in found_movies:
        print(f"movie_id: {movie_id}, title: {title}") #Print as requested

In [6]:
# LEAVE AS-IS

# For testing, this should print:
# movie_id:  4993, title: Lord of the Rings: The Fellowship of the Ring, The (2001)
# movie_id:  5952, title: Lord of the Rings: The Two Towers, The (2002)
# movie_id:  7153, title: Lord of the Rings: The Return of the King, The (2003)
find_movies("Lord of the Rings", movies)

movie_id: 4993, title: Lord of the Rings: The Fellowship of the Ring, The (2001)
movie_id: 5952, title: Lord of the Rings: The Two Towers, The (2002)
movie_id: 7153, title: Lord of the Rings: The Return of the King, The (2003)


The following function, which you can leave as-is, prints the title of a movie given its movie_id.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [7]:
# LEAVE AS-IS

def get_title(movie_id, movies):
    return movies[movies['movie_id'] == movie_id].title.iloc[0]

In [8]:
# LEAVE AS-IS

# For testing, should print "Lord of the Rings: The Return of the King, The (2003)")
print(get_title(7153, movies))

Lord of the Rings: The Return of the King, The (2003)


## 1.3. Count unique registers

Count the number of unique users and unique movies in the `ratings` variable. Use [unique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html). Print also the total number of movies in the `movies` variable. Your code should print:

```
Number of users who have rated a movie : 12676
Number of movies that have been rated  : 2049
Total number of movies                 : 33168
```

Note that ratings are heavily concentrated on a few popular movies.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your own code to indicate the number of unique users and unique movies in the "ratings" variable.</font>

In [9]:
# Number of unique users in ratings
num_users = ratings['user_id'].nunique()

# Number of unique movies in ratings
num_movies_rated = ratings['movie_id'].nunique()

# Total number of movies in movies
total_movies = movies['movie_id'].nunique()

print(f"Number of users who have rated a movie: {num_users}")
print(f"Number of movies that have been rated: {num_movies_rated}")
print(f"Total number of movies: {total_movies}")

Number of users who have rated a movie: 12676
Number of movies that have been rated: 2049
Total number of movies: 33168


# 2. Item-based Collaborative Filtering

The two main types of interactions-based recommender system, also known as *collaborative filtering* algorithms are:

1. **User-based Collaborative Filtering**: To recommend items for user A, we first look at other users B1, B2, ..., Bk with a similar behavior to A, and aggregate their preferences. For instance, if all Bi like a movie that A has not watched, it would be a good candidate to be recommended. 


2. **Item-based Collaborative Filtering**: To recommend items for user A, we first look at all the items I1, I2, ..., Ik that the user A has consumed, and find items that elicit similar ratings from other users. For instnce, an item that is rated positively by the same users that rate positively the Ii items, and negatively by the same users that rate negatively the Ii items, would be a good candidate to be recommended.

In both cases, a similarity matrix needs to be built. For user-based, the **user-similarity matrix** will consist of some **distance metrics** that measure the similarity between any two pairs of users. For item-based, the **matrix** will measure the similarity between any two pairs of items.

As we already know, there are several metrics strategy for measure the "similarity" of two items. Some of the most used metrics are Jaccard, Cosine and Pearson. Meanwhile, Jaccard similarity is based on the number of users which have rated item A and B divided by the number of users who have rated either A or B (very useful for those use cases where there is not a numeric rating but just a boolean value like a product being bought), in Pearson and Cosine similarities we measure the similarity between two vectors.

For the purpose of this assignment, we will use **Pearson Similarity** and we will implement a **Item-based Collaborative filtering**.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

## 2.1. Data pre-processing

Firstly, create a new dataframe called "rated_movies" that is simply the "ratings" dataset with column genres removed using the [Drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) function.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to generate "rated_movies" and print the first ten rows. This should have columns user_id, movie_id, rating, title</font>

In [10]:
rated_movies = ratings.drop('genres', axis=1)

# Print the first ten rows of rated_movies
rated_movies.head(10)

,user_id,movie_id,rating,title
0,4,3624,2.5,Shanghai Noon (2000)
1,152,3624,3.0,Shanghai Noon (2000)
2,171,3624,3.5,Shanghai Noon (2000)
3,276,3624,4.0,Shanghai Noon (2000)
4,494,3624,3.5,Shanghai Noon (2000)
5,1148,3624,2.5,Shanghai Noon (2000)
6,1967,3624,2.0,Shanghai Noon (2000)
7,2189,3624,4.0,Shanghai Noon (2000)
8,2287,3624,4.0,Shanghai Noon (2000)
9,2360,3624,4.0,Shanghai Noon (2000)


Now, using the `rated_movies` dataframe, create a new dataframe named `ratings_summary` containing the following columns:

* movie_id
* title
* ratings_mean (average rating)
* ratings_count (number of people who have rated this movie)

You can use the following operations:

* Initialize `ratings_summary` to be only the movie_id and title of all movies in `rated_movies`
   * To group dataframe `df` by column `a` and keep only one unique row per value of `a`, use: `df.groupby('a').first()`
* Compute two series: `ratings_mean` and `ratings_count`:
   * To obtain a series with the average of column `a` for each distinct value of column `b` in dataframe `df`, use: `df.groupby(b)['a'].mean()`
   * To obtain a series with the count of column `a` for each distinct value of column `b` in dataframe `df`, use: `df.groupby(b)['a'].count()`
* Add these series to the `ratings_summary`
   * To add a series `s` with column name `a` to dataframe `df`, use: `df['a'] = s`
    
<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to generate "ratings_summary" and print the first 5 rows.</font>

In [11]:
# Initialize ratings_summary with movie_id and title
ratings_summary = rated_movies[['movie_id', 'title']].groupby('movie_id').first()

# Compute ratings_mean and ratings_count
ratings_summary['ratings_mean'] = rated_movies.groupby('movie_id')['rating'].mean()
ratings_summary['ratings_count'] = rated_movies.groupby('movie_id')['rating'].count()

# Print the first 5 rows of ratings_summary
ratings_summary.head(5)

,title,ratings_mean,ratings_count
movie_id,,,
2769,"Yards, The (2000)",3.122549,102
3177,Next Friday (2000),2.824000,125
3190,Supernova (2000),2.395683,139
3225,Down to You (2000),2.577273,110
3228,Wirey Spindell (2000),2.500000,2


To select from dataframe A those having column C larger or equal to N, you can do `A[A.C >= N]`.

To sort dataframe A by decreasing values of column C, you can do `A.sort_values(by='C', ascending=False)`.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to print the top 5 highest rated movies, considering only movies receiving at least 2500 ratings.</font>

In [12]:
# Filter movies with at least 2500 ratings
high_rated_movies = ratings_summary[ratings_summary['ratings_count'] >= 2500]

# Sort the filtered dataframe by decreasing values of ratings_mean
top_rated_movies = high_rated_movies.sort_values(by='ratings_mean', ascending=False)

# Print the top 5 highest-rated movies
top_rated_movies.head(5)

,title,ratings_mean,ratings_count
movie_id,,,
4226,Memento (2000),4.158512,4476
4973,"Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le)...",4.097234,3687
4993,"Lord of the Rings: The Fellowship of the Ring,...",4.092530,5944
7153,"Lord of the Rings: The Return of the King, The...",4.083960,5449
5952,"Lord of the Rings: The Two Towers, The (2002)",4.083869,5449


<font size="+1" color="red">Repeat this, but this time consider movies receiving at least 3 ratings. What is the difference? How would you explain this?</font>

In [13]:
# Filter movies with at least 3 ratings
high_rated_3_ratings = ratings_summary[ratings_summary['ratings_count'] >= 3]

# Sort the filtered dataframe by decreasing values of ratings_mean
top_rated_3_ratings = high_rated_3_ratings.sort_values(by='ratings_mean', ascending=False)

# Print the top 5 highest-rated movies with at least 3 ratings
top_rated_3_ratings.head(5)

,title,ratings_mean,ratings_count
movie_id,,,
5082,"Rumor of Angels, A (2000)",4.666667,6
27764,2LDK (2003),4.500000,3
31954,Beautiful City (Shah-re ziba) (2004),4.400000,5
5224,Promises (2001),4.388889,18
6775,Life and Debt (2001),4.333333,3


The main difference beetween the mean of the ratings of films with more that 2500 ratings, and films with at least more than 3 ratings is that in the second case the value of the top 5 highest rated films is higher. This fact can occur because with less ratings the value of the mean is less accurate in respect of the quality of the film. So if we observe the highest rating films there appear films that are better valued than they should be because of the lack of ratings. This phenomenom can also apply to the mean of the ratings in a negative way, if we see the lowest rated films with at least 3 ratings, we can observe that there are films unfairly rated because of the same thing that I mentioned before. In conclusion the films with more ratings have a more realiable value of the ratings mean.

## 2.2. Compute the user-movie matrix

Before calculating the **similarity matrix**, we create a table where columns are movies and rows are users, and each movie-user cell contains the score of that user for that movie.

We will use the [pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) function of Pandas, which receives a dataframe plus the variable that will make the rows, the variable that will make the columns, and the variable that will make the cells, and transform it into a matrix of the specified rows, columns, and cells.

For instance, if you have a dataframe D containing:

```
U V W
1 a 3.0
1 b 2.0
2 a 1.0
2 c 4.0
```

Calling `D.pivot_table(index='U', columns='V', values='W')` will create the following:

```
V  a   b   c
U
1 3.0 2.0 NaN
2 1.0 NaN 4.0
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to generate a "user_movie" matrix by calling "pivot_table" on "rated_movies". Print the first 5 rows. It might take about one minute to compute, depending on your computer.</font>

In [14]:
# Create the user_movie matrix using pivot_table
user_movie = rated_movies.pivot_table(index='user_id', columns='movie_id', values='rating')

# Print the first 5 rows of the user_movie matrix
print(user_movie.head(5))

movie_id  2769   3177   3190   3225   3228   3239   3273   3275   3276   \
user_id                                                                   
4           NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
62          NaN    NaN    NaN    NaN    NaN    NaN    NaN    4.5    NaN   
63          NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
95          NaN    NaN    NaN    NaN    NaN    NaN    NaN    3.5    NaN   

movie_id  3279   ...  33138  33145  33148  33150  33152  33154  33158  33162  \
user_id          ...                                                           
4           NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
33          NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
62          NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
63          NaN  ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    N

<font size="+1" color="red">Replace this a brief commentary indicating why do you think the "user_movie" matrix has so many "NaN" values. How do we call this characteristic of user ratings in recommender systems?</font>

The "user_movie" matrix has many "NaN" values because not every user has rated every movie. This characteristic of user ratings in recommender systems is known as sparsity, and it occurs when the majority of the entries in the user-item interaction matrix are missing, as users typically rate or interact with only a small fraction of the available items.

# 2.3. Explore some correlations in the user-movie matrix

Now let us explore whether correlations in this matrix make sense.

1. Locate the movie_id for the following three movies:
  * [Lord of the Rings: The Fellowship of the Ring (2001)](https://en.wikipedia.org/wiki/The_Lord_of_the_Rings:_The_Fellowship_of_the_Ring) -- name this id_pivot
  * [Finding Nemo (2003)](https://en.wikipedia.org/wiki/Finding_Nemo) -- name this id_m1
  * [Talk to Her (Hable con Ella) (2002)](https://en.wikipedia.org/wiki/Talk_to_Her) -- name this id_m2
2. Obtain the ratings for each of these movies: `user_movie[movie_id].dropna()`. You will obtain a column, containing a series of ratings for each movie.
3. Consolidate these four series into a single dataframe: `ratings3 = pd.concat([s1, s2, s3], axis=1)`
4. Drop from `ratings3` all rows containing a *NaN*. This will keep only the users that have rated all the 3 movies.
5. Display the first 10 rows from this table.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to compute and display the first 10 rows of the "ratings3" table as described above.</font>

In [15]:
# Locate the movie_id for the following three movies
id_pivot = movies[movies['title'] == 'Lord of the Rings: The Fellowship of the Ring, The (2001)']['movie_id'].values
id_m1 = movies[movies['title'] == 'Finding Nemo (2003)']['movie_id'].values
id_m2 = movies[movies['title'] == 'Talk to Her (Hable con Ella) (2002)']['movie_id'].values

# Obtain the ratings for each of these movies
s1 = user_movie[id_pivot].dropna()
s2 = user_movie[id_m1].dropna()
s3 = user_movie[id_m2].dropna()

# Consolidate these four series into a single dataframe
ratings3 = pd.concat([s1, s2, s3], axis=1)
# Drop from ratings3 all rows containing a NaN
ratings3 = ratings3.dropna()

# Display the first 10 rows from this table
ratings3.head(10)

movie_id,4993,6377,5878
user_id,,,
859,3.0,4.0,5.0
1229,4.0,4.0,4.5
1281,3.0,2.5,3.0
1722,5.0,4.5,4.0
2004,4.5,3.0,3.5
4590,4.0,4.0,2.0
5052,2.0,4.0,4.0
5144,5.0,5.0,5.0
6497,3.5,3.5,3.5


To compute Pearson correlation, we use the [corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.corr.html) method.

To compute the correlation between two columns `a`, `b` in dataframe `df`, we use: `df[a].corr(df[b])`.

Compute the correlations between all pairs of columns of the `ratings3` table. You should display:

```
Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Finding Nemo (2003)': 0.38
Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Talk to Her (Hable con Ella) (2002)': 0.16
Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': 0.20
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to compute all correlations between these three movies, as described above.</font>

In [16]:
# Compute correlations between all pairs of columns in ratings3
correlation_lotr_nemo = ratings3[id_pivot[0]].corr(ratings3[id_m1[0]])
correlation_lotr_talk = ratings3[id_pivot[0]].corr(ratings3[id_m2[0]])
correlation_nemo_talk = ratings3[id_m1[0]].corr(ratings3[id_m2[0]])

# Display the computed correlations
print(f"Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Finding Nemo (2003)': {correlation_lotr_nemo:.2f}")
print(f"Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Talk to Her (Hable con Ella) (2002)': {correlation_lotr_talk:.2f}")
print(f"Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': {correlation_nemo_talk:.2f}")

Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Finding Nemo (2003)': 0.38
Similarity between 'Lord of the Rings: The Fellowship of the Ring, The (2001)' and 'Talk to Her (Hable con Ella) (2002)': 0.16
Similarity between 'Finding Nemo (2003)' and 'Talk to Her (Hable con Ella) (2002)': 0.20


<font size="+1" color="red">Replace this cell with a brief commentary on the correlations you find.</font>

On the one hand the similarity between the 'Lord of the Rings' film and 'Finding Nemo' is 0.38. This suggests a moderate positive correlation, indicating that users who enjoyed one of these movies tended to also enjoy the other to some extent. On the other hand the similarity between 'Talk to Her' and the 'Lord of the Rings' film is 0.16. This correlation is lower, suggesting a weaker positive correlation between user ratings for these two movies. Users who enjoyed one of these movies were less likely to have a strong preference for the other. Finally, the similarity beetween 'Finding Nemo' and 'Talk to Her' is 0.20. This correlation indicates a slightly positive correlation, where users who enjoyed 'Finding Nemo' were somewhat likely to also to enjoy 'Talk to Her'.

Now let us take the first movie selected above, the one with movie_id `id_pivot`.

Select the column corresponding to this movie in `user_movies` and compute its correlation with all other columns in `user_movies`. This can be done with  [corrwith](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corrwith.html).

*Note 1*: You might receive a runtime warning on degrees of freedom and/or division by zero, which you can safely ignore. It simply means that there are some columns that have no elements in common with the given column, or only one element in common, and thus for which the correlation cannot be computed.

*Note 2*: Note that the similarities that you computed above with `corr` are limited to the set of users who rated all the three movies. Instead, the similarities that you compute below with `corrwith` include all users who rated the pivot plus at least one other movie. Hence, the results could be different.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to create a "similar_to_pivot" series that contains the computed correlations, droping the NaNs in the series.</font>

In [17]:
# Compute correlations with the movie_id equal to id_pivot
similar_to_pivot = user_movie.corrwith(user_movie[id_pivot[0]])

# Drop NaN values in the series
similar_to_pivot = similar_to_pivot.dropna()

# Display the computed correlations
print(similar_to_pivot)

C:\Users\berna\anaconda3\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
C:\Users\berna\anaconda3\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


movie_id
2769    -0.127515
3177     0.093221
3190     0.041206
3225     0.126600
3239     0.338378
           ...   
33154    0.318255
33158    0.228214
33162    0.285377
33164    0.037130
33166    0.197344
Length: 1868, dtype: float64


Next, create a dataframe `corr_with_pivot` by using `similar_to_pivot` and `ratings_summary`. This dataframe should have the following columns:

* corr - the correlation between each movie and the selected movie
* title
* ratings_mean
* ratings_count

To create a dataframe `df` from a series `s`, use: `df = pd.DataFrame(s, columns=['colname'])`. 

Keep only rows in which *ratings_count* > 500, i.e., popular movies. To filter a dataframe `df` and keep only rows having column `c` larger than `x`, use `df[df[c] > x]`.

Display the top 10 rows with the largest correlation. To select the largest `n` rows from dataframe `df` according to column `c`, use `df.sort_values(c, ascending=False).head(n)`. 

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to create a "corr_with_pivot" dataframe as specified above, and to print the 10 movies (rated 500 times or more) with the highest correlation with the selected movie.</font>

In [18]:
# Create a dataframe corr_with_pivot
corr_with_pivot = pd.DataFrame(similar_to_pivot, columns=['corr'])
corr_with_pivot['title'] = ratings_summary['title']
corr_with_pivot['ratings_mean'] = ratings_summary['ratings_mean']
corr_with_pivot['ratings_count'] = ratings_summary['ratings_count']

# Keep onlym ovies with ratings_count > 500
corr_with_pivot = corr_with_pivot[corr_with_pivot['ratings_count'] > 50]

# Display the top 10 rows with the largest correlation
top_correlated_movies = corr_with_pivot.sort_values('corr', ascending=False).head(10)
top_correlated_movies.head(10)

,corr,title,ratings_mean,ratings_count
movie_id,,,,
4993,1.000000,"Lord of the Rings: The Fellowship of the Ring,...",4.092530,5944
5952,0.892103,"Lord of the Rings: The Two Towers, The (2002)",4.083869,5449
7153,0.892073,"Lord of the Rings: The Return of the King, The...",4.083960,5449
5485,0.506368,Tadpole (2002),3.225806,62
5686,0.460638,Russian Ark (Russkiy Kovcheg) (2002),3.084906,53
6185,0.455579,Dark Blue (2003),3.254717,53
6868,0.442703,Wonderland (2003),3.216667,60
5297,0.439536,"Cat's Meow, The (2002)",2.942308,52
31437,0.435456,Nobody Knows (Dare mo shiranai) (2004),3.914286,70


<font size="+1" color="red">Replace this cell with a brief commentary about the movies you see on this list. What happens if you set the condition on *ratings_count* to a much larger value? What happens if you set it to a much smaller value?</font>

If I set the condition ratings_count to a much larger value the dataframe does not vary because setting the condition at 500 already sets the ratings_count to high numbers having more correlation, which makes sense because with more ratings appear films with higher correlation regarding movies with less ratings. If I set the condition to a much smaller value, the three first Lord of the Rings films are still the three top highest correlated movies, but the the other movies have a low number of ratings and high correlation.

# 2.4. Implement the item-based recommendations

Now that we believe that this type of correlation sort of makes sense, let us implement the item-based recommender. We need all correlations between columns in `user_movie`.

To compute all correlations between columns in a dataframe, use [corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html). This function receives a matrix with *r* rows and *c* columns, and returns a square matrix of *c x c* containing all pair-wise correlations between columns.

**This process may take a few minutes.** Print the first 5 rows of the resulting matrix when done.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to compute all correlations between columns (movies) in the matrix user_movie. Store this in "item_similarity", and print the first 10 rows.</font>

In [19]:
# Compute all correlations between columns in user_movie
item_similarity = user_movie.corr()

# Print the first 10 rows of item_similarity
item_similarity.head(10)

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
movie_id,,,,,,,,,,,,,,,,,,,,,
2769,1.000000,0.115068,0.033721,-0.232268,NaN,-0.500000,0.197011,0.199514,0.250873,NaN,...,0.379980,0.878310,NaN,NaN,NaN,0.248126,1.806095e-01,-0.085570,-0.408248,0.105671
3177,0.115068,1.000000,0.303820,0.559533,NaN,NaN,0.331191,0.167918,1.000000,NaN,...,0.546119,0.735767,-1.0,NaN,NaN,-0.221382,3.174747e-01,0.014735,0.661989,0.185654
3190,0.033721,0.303820,1.000000,0.636361,NaN,-0.014315,0.146042,0.394293,-0.290397,NaN,...,0.246183,0.632026,NaN,NaN,NaN,0.378181,1.709261e-01,0.022444,-0.073360,-0.054114
3225,-0.232268,0.559533,0.636361,1.000000,NaN,0.578414,0.347716,0.263671,-0.250313,NaN,...,-0.300376,0.318377,NaN,NaN,NaN,0.480173,7.503063e-01,0.536828,0.753141,0.098748
3228,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3239,-0.500000,NaN,-0.014315,0.578414,NaN,1.000000,0.180846,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,1.000000,0.636285,0.888200
3273,0.197011,0.331191,0.146042,0.347716,NaN,0.180846,1.000000,0.105735,0.154371,NaN,...,0.006774,0.409968,1.0,NaN,NaN,0.088405,7.516779e-02,0.143492,0.466705,0.084202
3275,0.199514,0.167918,0.394293,0.263671,NaN,1.000000,0.105735,1.000000,0.485071,NaN,...,-0.011426,0.279624,NaN,NaN,NaN,0.075827,2.994603e-01,0.187713,0.285584,0.225317
3276,0.250873,1.000000,-0.290397,-0.250313,NaN,NaN,0.154371,0.485071,1.000000,NaN,...,NaN,0.292770,NaN,NaN,NaN,0.000000,-6.885311e-17,-0.455530,0.500000,-0.138013


Similarities between movies that do not have many ratings in common are unreliable. Fortunately, the `corr` method includes a parameter `min_periods` that establishes a minimum number of elements in common that two columns must have to compute the correlation.

Re-generate item_similarity setting min_periods to 100.

This process will also take a few minutes. Print the first 5 rows of the resulting matrix when done.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to compute all correlations between columns (movies) in the matrix user_movie, but considering only movies having at least 100 ratings in common. Store this in "item_similarity_min_ratings"</font>

In [20]:
# Compute all correlations between columns in user_movie with min_periods set to 100
item_similarity_min_ratings = user_movie.corr(min_periods=100)

# Print the first 5 rows of item_similarity_min_ratings
item_similarity_min_ratings.head(10)

movie_id,2769,3177,3190,3225,3228,3239,3273,3275,3276,3279,...,33138,33145,33148,33150,33152,33154,33158,33162,33164,33166
movie_id,,,,,,,,,,,,,,,,,,,,,
2769,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3177,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3190,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3225,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3273,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.105735,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.084202
3275,NaN,NaN,NaN,NaN,NaN,NaN,0.105735,1.000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.187713,NaN,0.225317
3276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will need to test our function so let us select a couple of interesting users.

Our first user, `user_id_super` will be someone who has given the following 3 films a rating higher than 4.5:

* movie_id=5349: *Spider-Man (2002)*
* movie_id=3793: *X-Men (2000)*
* movie_id=6534: *Hulk (2003)* 	

Our second user, `user_id_drama` will be someone who has given the following 3 films a rating higher than 4.5:

* movie_id=6870: *Mystic River (2003)*
* movie_id=5995: *Pianist, The (2002)*
* movie_id=3555: *U-571 (2000)*

To filter a dataframe by multiple conditions you can use, e.g., `df[(a > 1) & (b > 2)]`. 

**Important**: these particular users have watched lots of movies, so we cannot tell for sure they have only these interests.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to find the userids of two example users: user_id_super (the who liked the three superhero movies), and user_id_drama (the one who liked the three dramas)</font>

In [25]:
# Find user_id_super who liked the three superhero movies
user_id_super = ratings_raw[(ratings_raw['movie_id'].isin([5349, 3793, 6534])) & (ratings_raw['rating'] > 4.5)]['user_id']

# Find user_id_drama who liked the three drama movies
user_id_drama = ratings_raw[(ratings_raw['movie_id'].isin([6870, 5995, 3555])) & (ratings_raw['rating'] > 4.5)]['user_id']

# Print the userids
print(type(user_id_super))
print(type(user_movie))
print("user_id_super:\n",user_id_super)
print("\nuser_id_drama:\n",user_id_drama)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
user_id_super:
 10240        1030
10630        1041
15405        1458
19483        1881
25050        2288
            ...  
2151426    161005
2151432    161005
2153744    161212
2160065    161771
2161799    162018
Name: user_id, Length: 574, dtype: int64

user_id_drama:
 2951          276
9627          859
9659          859
10697        1051
11825        1162
            ...  
2163518    162193
2164046    162232
2164806    162249
2165670    162297
2166351    162436
Name: user_id, Length: 578, dtype: int64


We will need some auxiliary functions that are provided below. You can leave as-is.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [41]:
# Leave this code as-is

# Gets a list of watched movies for a user_id
def get_watched_movies(user_id, user_movie):
    return list(user_movie.loc[user_id.values[0]].dropna().sort_values(ascending=False).index)
    
# Gets the rating a user_id has given to a movie_id
def get_rating(user_id, movie_id, user_movie):
    rating_series = user_movie[movie_id][user_id]
    if len(rating_series) == 1:
        return rating_series.item()
    else:
        # Handle multiple ratings (e.g., take the mean)
        return rating_series.mean()
# Print watched movies
def print_watched_movies(user_id, user_movie, movies):
    for movie_id in get_watched_movies(user_id, user_movie):
        print("%d %.1f %s " %
          (movie_id, get_rating(user_id, movie_id, user_movie), get_title(movie_id, movies)))

In [42]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_super, user_movie, movies)

3408 4.0 Erin Brockovich (2000) 
3996 4.0 Crouching Tiger, Hidden Dragon (Wo hu cang long) (2000) 
4973 4.1 Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001) 
5349 4.6 Spider-Man (2002) 
5952 4.5 Lord of the Rings: The Two Towers, The (2002) 
5816 4.3 Harry Potter and the Chamber of Secrets (2002) 
5810 3.8 8 Mile (2002) 
5507 3.6 xXx (2002) 
5481 3.6 Austin Powers in Goldmember (2002) 
5418 4.3 Bourne Identity, The (2002) 
5377 4.2 About a Boy (2002) 
4054 3.5 Save the Last Dance (2001) 
4995 4.2 Beautiful Mind, A (2001) 
4993 4.5 Lord of the Rings: The Fellowship of the Ring, The (2001) 
5989 4.2 Catch Me If You Can (2002) 
4865 3.5 From Hell (2001) 
4776 3.9 Training Day (2001) 
4720 4.1 Others, The (2001) 
4306 4.2 Shrek (2001) 
4246 3.8 Bridget Jones's Diary (2001) 
5991 3.9 Chicago (2002) 
4018 3.6 What Women Want (2000) 
4034 4.1 Traffic (2000) 
4014 3.9 Chocolat (2000) 
4022 3.9 Cast Away (2000) 
5064 4.1 The Count of Monte Cristo (2002) 
3967 4.1 Billy Elliot (2000) 
5013 

In [43]:
# LEAVE AS-IS (TESTING CODE)

print_watched_movies(user_id_drama, user_movie, movies)

3510 4.0 Frequency (2000) 
3555 4.3 U-571 (2000) 
3354 2.9 Mission to Mars (2000) 
3408 3.9 Erin Brockovich (2000) 
3624 3.6 Shanghai Noon (2000) 
3753 3.7 Patriot, The (2000) 
3481 4.0 High Fidelity (2000) 
3513 3.7 Rules of Engagement (2000) 


For every user, we will consider that the importance of a new movie (a movie s/he has not rated) will be equal to the sum of the similarities between that new movie and all the movies the user has already rated.

Indeed, to further improve this, we will compute a weighted sum, in which the weight will be the rating given to the movie.

For instance, suppose a user has rated movies as follows:

```
movie_id rating
1        2.0
2        3.0
3        NaN
4        NaN
```

And that movie similarities are as follows (values with a "." do not matter in this example):

```
movie_id   1   2   3   4
1         ...............
2         ...............
3         0.1 0.2 NaN ...
4         0.9 0.8 ... NaN
```

The importance of movie 3 to this user will be:

```
2.0 * 0.1 + 3.0 * 0.2 = 0.8
```

While the importance of movie 4 to this user will be:

```
2.0 * 0.9 + 3.0 + 0.8 = 5.6
```

As we can see, we are favoring movies that are highly similar to many movies that the user has rated high.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Create a function `get_movies_relevance` that returns a dataframe with columns `movie_id` and `relevance`. You can use the following template:

```python
def get_movies_relevance(user_id, user_movie, item_similarity_matrix):
    
    # Create an empty series
    movies_relevance = ...
    
    # Iterate through the movies the user has watched
    for watched_movie in ...
        
        # Obtain the rating given
        rating_given = ...
        
        # Obtain the vector containing the similarities of watched_movie
        # with all other movies in item_similarity_matrix
        similarities = ...
        
        # Multiply this vector by the given rating
        weighted_similarities = ...
        
        # Append these terms to movies_relevance
        movies_relevance = movies_relevance.append(weighted_similarities)
    
    # Compute the sum for each movie
    movies_relevance = movies_relevance.groupby(movies_relevance.index).sum()
    
    # Convert to a dataframe
    movies_relevance_df = pd.DataFrame(movies_relevance, columns=['relevance'])
    movies_relevance_df['movie_id'] = movies_relevance_df.index
    
    return movies_relevance_df

```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for "get_movies_relevance"</font>

In [131]:
def get_movies_relevance(user_id, user_movie, item_similarity_matrix):
    # Create an empty dataframe
    movies_relevance_df = pd.DataFrame(columns=['relevance', 'movie_id'])

    # Get only the movies that the user has rated
    user_id = user_id.tolist()
    rated_movies = user_movie.loc[user_id]

    # Iterate through the movies the user has watched
    for watched_movie, rating_given in rated_movies.iteritems():
        # Obtain the vector containing the similarities of watched_movie
        # with all other movies in item_similarity_matrix
        similarities = item_similarity_matrix[watched_movie]

        # Multiply this vector by the given rating
        weighted_similarities = similarities * rating_given

        # Create a dataframe for the current movie's relevance
        movie_relevance_df = pd.DataFrame({'relevance': weighted_similarities, 'movie_id': watched_movie})

        # Concatenate it to the main dataframe
        movies_relevance_df = pd.concat([movies_relevance_df, movie_relevance_df],ignore_index=True)

    # Compute the sum for each movie
    movies_relevance_df = movies_relevance_df.groupby('movie_id').sum()

    return movies_relevance_df

Apply `get_movies_relevance` to the two users we have selected, `user_id_super` and `user_id_drama`.

The result will contain only `movie_id` and `relevance`, you will have to merge with the `movies` dataframe on the `movie_id` attribute.

Sort the results by descending relevance and print the top 10 for each case.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to obtain the 5 most relevant movies for the users user_id_super (who likes superhero movies) and user_id_drama (who likes dramas)</font>

In [132]:
# Get the 5 most relevant movies for user_id_super
movies_relevance_super = get_movies_relevance(user_id_super, user_movie, item_similarity)
top_movies_super = movies_relevance_super.nlargest(5, 'relevance')

# Get the 5 most relevant movies for user_id_drama
movies_relevance_drama = get_movies_relevance(user_id_drama, user_movie, item_similarity)
top_movies_drama = movies_relevance_drama.nlargest(5, 'relevance')

# Display the results
print("Top 5 most relevant movies for user_id_super:")
print(top_movies_super)

print("\nTop 5 most relevant movies for user_id_drama:")
print(top_movies_drama)

Top 5 most relevant movies for user_id_super:
          relevance
movie_id           
3949       5.559356
7153       5.500308
4993       3.598786
5349       3.470298
3948       3.317576

Top 5 most relevant movies for user_id_drama:
          relevance
movie_id           
4223       2.066428
4019       1.832909
5152       1.780495
4995       1.663994
7458       1.568547


<font size="+1" color="red">Replace this cell with a brief commentary on the movies you see on these lists. How many of them look relevant for the intended users? Feel free to use IMDB or Wikipedia to get info on these movies.</font>

<font size="-1" color="gray">All those trivial facts you learned about 1980s and 1990s pop culture were supposed to be useful one day; that day has arrived :-)</font>

Finally, you only need to remove the movies the user has watched. To do so:

* Obtain the dataframe of relevant movies with `get_movies_relevance`
* Set this dataframe index to 'movie_id'
* Obtain the list of movie_ids of watched movies with `get_watched_movies`
* Drop from the relevant movies dataframe the watched movies

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code implementing "get_recommended_movies"</font>

In [143]:
def get_recommended_movies(user_id, user_movie, item_similarity_matrix):
    # Get the movies relevance dataframe
    movies_relevance_df = get_movies_relevance(user_id, user_movie, item_similarity_matrix)

    # Set the dataframe index to 'movie_id'
    movies_relevance_df.set_index('movie_id', inplace=True)
    print(movies_relevance)
    # Get the list of movie_ids of watched movies
    watched_movies = get_watched_movies(user_id, user_movie)

    # Drop the watched movies from the relevant movies dataframe
    recommended_movies_df = movies_relevance_df.drop(watched_movies, errors='ignore')

    return recommended_movies_df

<font size="+1" color="red">Replace this cell with your code to obtain the 10 most recommended movies for the users user_id_super and user_id_drama</font>

In [144]:
# Get recommended movies for user_id_super
user_id_super
recommended_movies_super = get_recommended_movies(user_id_super, user_movie, item_similarity)
recommended_movies_super.head(10)

KeyError: "None of ['movie_id'] are in the columns"

In [ ]:
# Get recommended movies for user_id_drama
recommended_movies_drama = get_recommended_movies(user_id_drama, user_movie, item_similarity)
recommended_movies_drama.head(10)

<font size="+1" color="red">Replace this cell with a brief commentary on these recommendations. Do you think they are relevant? Why or why not? After removing the movies the user has already watched, are the relevance scores of the remaining items comparable to the previous lists that contained all relevant movies?</font>

# DELIVER (individually)

Remember to read the section on "delivering your code" in the [course evaluation guidelines](https://github.com/chatox/data-mining-course/blob/master/upf/upf-evaluation.md).

Deliver a zip file containing:

* This notebook

## Extra points available

For more learning and extra points, use the [surprise](http://surpriselib.com/) library to generate recommendations for the same two users. Display the generated recommendations and comment on them.

**Note:** if you go for the extra points, add ``<font size="+2" color="blue">Additional results: surprise library</font>`` at the top of your notebook.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>